In [1]:
!cat .env

STAGE=PRODUCTION


In [2]:
import json
from django.contrib.admin.utils import flatten
from helpers.strings import keys_to_snake_case, keys_to_camel_case
from rest_framework_json_api.utils import format_field_names
from tqdm.notebook import tqdm
from django.db.models import CharField
from django.db.models.functions import Cast

In [3]:
def generate_select_multiple_export(field):

    result={"cat":[]}    
    values=field["selectOptions"]["options"]

    for value in values:
        if field["apiName"] == "demandOrigin":
            result["cat"].append({"if":[{"in": 
                                     [value["value"], {"var": "formData."+field["apiName"]}]
                                    }, value["name"] + " ", ""]})
        else:
            result["cat"].append({"if":[{"in": 
                                     [value["value"], {"var": "formData."+field["apiName"]}]
                                    }, value["name"] + ", ", ""]})


    return result

In [4]:
def generate_select_export(field):
    result={"if":[]}    
    values=field["selectOptions"]["options"]
    
    for value in values:
        result["if"].append({"==":[{"var": "formData."+field["apiName"]}, value["value"]]})
        result["if"].append(value["name"])
        
    return result

In [5]:
def generate_export(field):
    result={"header": field["displayName"], "key": field["apiName"]}
    
    if field["dataType"]=="timestamp":
        result["isDate"]=True
    elif (field["dataType"]=="select") and ("options" in field["selectOptions"]):
        result["logic"]=generate_select_export(field)
    elif (field["dataType"]=="selectMultiple") and ("options" in field["selectOptions"]):
        result["logic"]=generate_select_multiple_export(field)
    elif field["dataType"]=="jsonLogic":
        result["logic"]=field["logic"]
    elif field["dataType"]=="float":
        result["style"]= {
                "num_fmt": "0.000"
                }
    
    
    return result

In [6]:
def generate_array_export(field, number=6):
    result_list=[]
    values=field["innerFields"]
    
    for value in values:
        
        if (value["dataType"] == "select") and ("options" in value["selectOptions"]):
            for index in range(1, number+1):            
                result={}
                result["header"]=field["displayName"] + " - " + value["displayName"] + " "  + str(index)
                result["key"]=value["apiName"]+str(index)
                result["logic"]={}
                result["logic"]["if"]=[]
                values_select=value["selectOptions"]["options"]

                for value_select in values_select:
                    result["logic"]["if"].append({"==":[{"var": "formData." + field["apiName"]+ "."+str(index-1) + "." +value["apiName"]}, value_select["value"]]})
                    result["logic"]["if"].append(value_select["name"])

                result_list.append(result)
        elif (value["dataType"] == "select") and ("reference" in value["selectOptions"]) and company.name == "CAJ":
            lista = [{"name":a['name'],"uuid":a['uuid_str']} for a in OccurrenceType.objects.filter(company=company,occurrence_kind=6).annotate(uuid_str=Cast("uuid",output_field=CharField())).values('name', 'uuid_str')]
            for index in range(1, number+1):            
                result={}
                result["header"]=field["displayName"] + " - " + value["displayName"] + " "  + str(index)
                result["key"]=value["apiName"]+str(index)
                result["logic"]={}
                result["logic"]["if"]=[]
#                 print(lista)
                values_select=lista

                for value_select in values_select:
                    result["logic"]["if"].append({"==":[{"var": "formData." + field["apiName"]+ "."+str(index-1) + "." +value["apiName"]}, value_select["uuid"]]})
                    result["logic"]["if"].append(value_select["name"])

                result_list.append(result)
        
        elif (value["dataType"] == "boolean") and ("display" in value["apiName"]):
            pass
        
        elif value["dataType"] == "innerImagesArray":
            pass
        
        elif (value["dataType"] == "selectMultiple") and ("options" in value["selectOptions"]):
            for index in range(1, number+1):            
                result={}
                result["header"]=field["displayName"] + " - " + value["displayName"] + " "  + str(index)
                result["key"]=value["apiName"]+str(index)
                result["logic"]={}
                result["logic"]["cat"]=[]
                values_select=value["selectOptions"]["options"]

                for value_select in values_select:
                    result["logic"]["cat"].append({"if":[{"in": 
                                     [value_select["value"], {"var": "formData." + field["apiName"]+ "."+str(index-1) + "." +value["apiName"]}]
                                    }, value_select["name"], ", "]})

                result_list.append(result)
            
        else:
            for index in range(1, number+1):
                result={}
                result["header"]=field["displayName"] + " - " + value["displayName"] + " " + str(index)
                result["key"]=value["apiName"]+str(index)
                result["logic"]= {"var": "formData." + field["apiName"] + "." + str(index-1) + "." + value["apiName"]}
                result_list.append(result)

    
    
    return result_list

In [7]:
def generate_array_export_lado_a_lado(field, number=6):
    result_list=[]
    values=field["innerFields"]
    
    for value in values:
        for index in range(1, number+1): 
            if (value["dataType"] == "select") and ("options" in value["selectOptions"]):            
                result={}
                result["header"]=field["displayName"] + " - " + value["displayName"] + " "  + str(index)
                result["key"]=value["apiName"]+str(index)
                result["logic"]={}
                result["logic"]["if"]=[]
                values_select=value["selectOptions"]["options"]

                for value_select in values_select:
                    result["logic"]["if"].append({"==":[{"var": "formData." + field["apiName"]+ "."+str(index-1) + "." +value["apiName"]}, value_select["value"]]})
                    result["logic"]["if"].append(value_select["name"])

            elif (value["dataType"] == "select") and ("reference" in value["selectOptions"]) and company.name == "CAJ":
                lista = [{"name":a['name'],"uuid":a['uuid_str']} for a in OccurrenceType.objects.filter(company=company,occurrence_kind=6).annotate(uuid_str=Cast("uuid",output_field=CharField())).values('name', 'uuid_str')]           
                result={}
                result["header"]=field["displayName"] + " - " + value["displayName"] + " "  + str(index)
                result["key"]=value["apiName"]+str(index)
                result["logic"]={}
                result["logic"]["if"]=[]
    #                 print(lista)
                values_select=lista

                for value_select in values_select:
                    result["logic"]["if"].append({"==":[{"var": "formData." + field["apiName"]+ "."+str(index-1) + "." +value["apiName"]}, value_select["uuid"]]})
                    result["logic"]["if"].append(value_select["name"])


            elif (value["dataType"] == "boolean") and ("display" in value["apiName"]):
                pass

            elif value["dataType"] == "innerImagesArray":
                pass

            elif (value["dataType"] == "selectMultiple") and ("options" in value["selectOptions"]):
                            
                result={}
                result["header"]=field["displayName"] + " - " + value["displayName"] + " "  + str(index)
                result["key"]=value["apiName"]+str(index)
                result["logic"]={}
                result["logic"]["cat"]=[]
                values_select=value["selectOptions"]["options"]

                for value_select in values_select:
                    result["logic"]["cat"].append({"if":[{"in": 
                                     [value_select["value"], {"var": "formData." + field["apiName"]+ "."+str(index-1) + "." +value["apiName"]}]
                                    }, value_select["name"], ", "]})


            else:
                result={}
                result["header"]=field["displayName"] + " - " + value["displayName"] + " " + str(index)
                result["key"]=value["apiName"]+str(index)
                result["logic"]= {"var": "formData." + field["apiName"] + "." + str(index-1) + "." + value["apiName"]}
            result_list.append(result)

    
    
    return result_list

In [8]:
def update_export_data(data, occ=None, filename=None):
    if occ is not None:
        if 'measurement_columns' in occ.form_fields.keys():
            del occ.form_fields['measurement_columns']
        occ.form_fields['measurementColumns']=data
        occ.save()
    if filename is not None:
        with open(filename + '.txt', 'w') as outfile:
            json.dump(data, outfile, indent=4)

In [9]:
def format_keys(obj, format_type="camelize"):
    if isinstance(obj, dict):
        for key, value in obj.items():
            if isinstance(value, dict):
                obj[key] = format_field_names(
                            format_keys(value), format_type
                            )
            elif isinstance(value, list):
                list_temp = value.copy()
                for i, item in enumerate(value):
                    if isinstance(item, dict):
                        list_temp[i] = format_field_names(
                        format_keys(item), format_type
                        )
                    else:
                        list_temp[i] = item
                obj[key] = list_temp
    return obj

In [10]:
# company=Company.objects.get(name='Eixo Sp - Pavimento')
company=Company.objects.get(name='Rota do Oeste - Obras')
company

<Company: c2bc7bad-0602-46d7-91ab-c26c5a8c406f: Rota do Oeste - Obras>

In [11]:
# occ=OccurrenceType.objects.get(uuid='f6033d7d-9649-420d-9125-2000aefcd25a')
occ=OccurrenceType.objects.filter(company=company,occurrence_kind__in=['31','32','33'], active=True, next_version__isnull=True).order_by('name')
# occ=OccurrenceType.objects.filter(company=company, active=True, next_version__isnull=True).order_by('name')
occ.count()

8

In [12]:
for match in tqdm(occ):
    for index, field in enumerate(match.form_fields['fields']):
        match.form_fields['fields'][index]=format_keys(keys_to_camel_case(field))
    for indexg, field_group in enumerate(match.form_fields['groups']):
        match.form_fields['groups'][indexg]=format_keys(keys_to_camel_case(field_group))
    match.save()

  0%|          | 0/8 [00:00<?, ?it/s]

In [13]:
field_group

{'order': 3, 'members': [6], 'displayName': 'Informações adicionais'}

In [14]:
# columns=[]

# for field in occ.form_fields['fields']:
    
#     if field['dataType'] != 'arrayOfObjects':
#         columns.append(generate_export(field))
#     else:
#         columns.append(generate_array_export(field))
    

# columns=flatten(columns)

# update_export_data(columns, occ=occ)

In [15]:
for match in tqdm(occ):
        
    try:
        columns=[]

        for field in match.form_fields['fields']:
            if field['dataType'] != 'arrayOfObjects':
                columns.append(generate_export(field))
            else:
#                 pass
                columns.append(generate_array_export(field, 6))
#                 columns.append(generate_array_export_lado_a_lado(field, 50))
                


        columns=flatten(columns)
        update_export_data(columns, occ=match)
    
    except Exception as e:
        print(f'{match.uuid} - {e}')

  0%|          | 0/8 [00:00<?, ?it/s]

In [23]:
columns

[{'header': 'Código de edital', 'key': 'contratualCode'},
 {'header': 'Origem',
  'key': 'demandOrigin',
  'logic': {'cat': [{'if': [{'in': ['1', {'var': 'formData.demandOrigin'}]},
      'Interna ',
      '']},
    {'if': [{'in': ['2', {'var': 'formData.demandOrigin'}]}, 'KCOR ', '']},
    {'if': [{'in': ['3', {'var': 'formData.demandOrigin'}]},
      'ARTESP DIN ',
      '']},
    {'if': [{'in': ['4', {'var': 'formData.demandOrigin'}]},
      'ARTESP DOP ',
      '']},
    {'if': [{'in': ['5', {'var': 'formData.demandOrigin'}]}, 'CCO ', '']},
    {'if': [{'in': ['6', {'var': 'formData.demandOrigin'}]},
      'EAF-Meio Ambiente ',
      '']},
    {'if': [{'in': ['7', {'var': 'formData.demandOrigin'}]},
      'EAF-Conservação ',
      '']},
    {'if': [{'in': ['8', {'var': 'formData.demandOrigin'}]},
      'EAF-Operação ',
      '']},
    {'if': [{'in': ['9', {'var': 'formData.demandOrigin'}]},
      'EAF-Vistoria ',
      '']},
    {'if': [{'in': ['10', {'var': 'formData.demandOrigin'

In [56]:
occ=OccurrenceType.objects.filter(uuid=str('da392f7a-ab26-47eb-900e-497ca86850c7')).first()
teste=occ.form_fields['fields'][5]

In [57]:
generate_array_export(teste)

[{'header': 'Inclusão de devolutivas - Classe 1',
  'key': 'occurrenceType1',
  'logic': {'if': [{'==': [{'var': 'formData.therapy.0.occurrenceType'},
      '029fcd2f-998b-4d19-802c-325617478c27']},
    'B2.3 - Sinalização',
    {'==': [{'var': 'formData.therapy.0.occurrenceType'},
      '02ed6a53-21ef-42e0-b9a8-703520ff54db']},
    'C3.3 - Integridade Das Peças',
    {'==': [{'var': 'formData.therapy.0.occurrenceType'},
      '078a0011-43b8-4345-ad31-3a16b010d87b']},
    'A1.1 - Controle Da Produção De Efluentes Contaminantes E Condições Sanitárias',
    {'==': [{'var': 'formData.therapy.0.occurrenceType'},
      '11e22bda-19e1-45f5-afcd-6744acba229e']},
    'B2.1 - Escoramento - Tem Projeto? Precisa Do Escoramento?',
    {'==': [{'var': 'formData.therapy.0.occurrenceType'},
      '1361cb22-a6f2-4eae-a233-b91be422915f']},
    'C2.6 - Espessura Das Camadas',
    {'==': [{'var': 'formData.therapy.0.occurrenceType'},
      '15f5f0a0-2be3-4ab2-9584-87442096701f']},
    'C3.6 - Viga De Tra

In [53]:
lista = [{"name":a['name'],"uuid":a['uuid_str']} for a in OccurrenceType.objects.filter(company=company,occurrence_kind=6).annotate(uuid_str=Cast("uuid",output_field=CharField())).values('name', 'uuid_str')]

In [54]:
lista

[{'name': 'B2.3 - Sinalização',
  'uuid': '029fcd2f-998b-4d19-802c-325617478c27'},
 {'name': 'C3.3 - Integridade Das Peças',
  'uuid': '02ed6a53-21ef-42e0-b9a8-703520ff54db'},
 {'name': 'A1.1 - Controle Da Produção De Efluentes Contaminantes E Condições Sanitárias',
  'uuid': '078a0011-43b8-4345-ad31-3a16b010d87b'},
 {'name': 'B2.1 - Escoramento - Tem Projeto? Precisa Do Escoramento?',
  'uuid': '11e22bda-19e1-45f5-afcd-6744acba229e'},
 {'name': 'C2.6 - Espessura Das Camadas',
  'uuid': '1361cb22-a6f2-4eae-a233-b91be422915f'},
 {'name': 'C3.6 - Viga De Travamento',
  'uuid': '15f5f0a0-2be3-4ab2-9584-87442096701f'},
 {'name': 'C3.1 - Execução Da Base - Areia',
  'uuid': '1a0fa90c-1f50-4eb1-9f96-46436ed5af61'},
 {'name': 'C5.1 - Reenquadro - Cortes Com Ângulos Retos',
  'uuid': '2590028b-06cc-4e05-b798-664886cc081d'},
 {'name': 'C2.4 - Umidade Do Material - Base E Sub-Base',
  'uuid': '2ffd8525-a322-4d32-94a5-27e1189b70fe'},
 {'name': 'B2.4 - Organização Do Canteiro/ Frente De Trabalho',

In [19]:
occ=[]
for a in Reporting.objects.filter(company=company):
    if a.name not in occ:
        occ.append(a.name)
    else:
        print(a.name)

In [19]:
for a in Reporting.objects.filter(company=company).values('occurrence_type__name').annotate(qtd_apt=Count('uuid')):
    print(a['occurrence_type__name'],a['qtd_apt'])

Desgaste 6
Camada Rolamento - Cbuq - Grad. C. - Com Dop 177
Fresagem 2248
Escorregamento  ARTESP 1
Trecho Rodoviário 3
Panela 32
Retrabalho 25
Trinca Interligada "Bloco" 6
Microrevestimento Asfaltico a Frio Com Polimero Sem Fibra 344
Depressão 18
Trinca Interligada "Jacaré" 27
Sinalização Hor. Tinta P/Pouco Trafego (Sin. Provisoria) 2331
Escorregamento ARTESP 1
Afundamento Da Trilha Plástico 9
Escorregamento 10
Estrutural 315
Ondulação Ou Corrugação 9
Camada Rolamento - Cbuq - Grad. 3 - Com Dop 760
Afundamento Local Plástico 7
Concreto Asfaltico Modificado por Polimero - Grad. 3 3
Trinca Interligada "Jacaré" ARTESP 1
Desplacamento 2
Imprimadura Betuminosa Ligante 934
Revestimento 2
Fresagem Continua de Pav. 253
Fresagem Fina 3
Reciclagem Capa/Base com Adição de 4% de Cimento 5
Afundamento Local Consolidado 57
Afundamento Da Trilha Consolidado 4
Fresagem Continua de Pav., Imprimadura Betuminosa Ligante, Camada Rolamento - CBUQ - Grad. 3 - Com Dop 1
Correção De Degrau 2
Fresagem Desconti

In [10]:
from tqdm.notebook import tqdm

In [17]:
for a in tqdm(Reporting.objects.filter(company=company).exclude(occurrence_type__name__in=['Fresagem','Desgaste','Estrutural'])):
    try:
        if len(a.form_data.get('service_dois')) >= 1:
            print(len(a.form_data.get('service_dois')),a.occurrence_type.name,a.number)
    except:
        pass

  0%|          | 0/5593 [00:00<?, ?it/s]

5 Afundamento Local Plástico EIXO-PAT-2023.00195
